In [1]:
from utils.data_preprocess import *
import pandas as pd
import numpy as np
import tensorflow as tf
import talib as ta

In [2]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')

In [4]:
raw_data = DataLabeling(df, volatility_period=12, alpha=0.55)

In [6]:
data = raw_data.TI_data

c:\Users\dddru\Documents\GitHub\Is-Technical-Indicator-Really-a-Thing-for-Deep-Learning\utils\data_preprocess.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  np.where(data.Trend > data.Trend.shift(-1), 'Sell', 'Hold'))


In [7]:
data.head()

,Open,High,Low,Close,Volume,Chaikin,Trange,Hammer,ShootingStar,RSI_5,...,RSI_30,DX_30,ADX_30,ADXR_30,EMA_30,BBand_upper_30,BBand_lower_30,DEMA_30,NATR_30,Label
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-07-14 11:20:00,0.088743,0.088743,0.088601,0.088741,2.364,352.962313,0.000198,100,0,50.046704,...,61.873485,21.657629,38.410184,46.304116,0.088611,0.089033,0.088521,0.089005,0.318941,Buy
2017-07-14 11:25:00,0.088743,0.088902,0.088612,0.088902,4.772,357.734313,0.000290,0,0,69.033911,...,62.927633,23.896195,37.926385,45.802243,0.088630,0.089035,0.088521,0.089016,0.318625,Sell
2017-07-14 11:30:00,0.088902,0.088902,0.088764,0.088902,2.902,360.636313,0.000138,0,0,69.033911,...,62.927633,23.896195,37.458712,45.295407,0.088648,0.089037,0.088520,0.089025,0.313178,Hold
2017-07-14 11:35:00,0.088902,0.088902,0.088773,0.088902,3.884,364.520313,0.000129,0,0,69.033911,...,62.927633,23.896195,37.006628,44.796801,0.088664,0.089045,0.088529,0.089032,0.307576,Sell
2017-07-14 11:40:00,0.088902,0.088902,0.088657,0.088657,1.533,362.987313,0.000245,0,0,32.414691,...,60.127013,21.101381,36.476453,44.263531,0.088664,0.089044,0.088520,0.089007,0.307356,Buy


In [305]:
data = df.set_index('Timestamp').iloc[:, :5]
data['STD'] = data.Close.rolling(12).std()
data['Next_Close'] = data.Close.shift(-1)
# data = data.dropna()
data.head()

,Open,High,Low,Close,Volume,STD,Next_Close
Timestamp,,,,,,,
2017-07-14 04:00:00,0.080000,0.080000,0.080000,0.080000,0.726,NaN,0.080001
2017-07-14 04:05:00,0.080000,0.080001,0.080000,0.080001,3.347,NaN,0.086400
2017-07-14 04:10:00,0.080001,0.086400,0.080001,0.086400,4.679,NaN,0.085128
2017-07-14 04:15:00,0.085289,0.085620,0.085128,0.085128,53.431,NaN,0.086000
2017-07-14 04:20:00,0.085274,0.086000,0.085274,0.086000,5.576,NaN,0.085811


In [306]:
data['Trend'] = np.where(data.Close.shift(-1) >= data.Close*(1+0.0001*data.STD), 1,
                         np.where(data.Close.shift(-1) <= data.Close*(1-0.0001*data.STD), -1, 0))
data = data.dropna()

In [307]:
data['Buy_Hold_Sell'] = np.where(data.Trend < data.Trend.shift(-1), 'Buy',
                                 np.where(data.Trend > data.Trend.shift(-1), 'Sell', 'Hold'))

In [308]:
data.head(50)

,Open,High,Low,Close,Volume,STD,Next_Close,Trend,Buy_Hold_Sell
Timestamp,,,,,,,,,
2017-07-14 04:55:00,0.086380,0.086380,0.086309,0.086347,6.691,0.002397,0.085978,-1,Hold
2017-07-14 05:00:00,0.085874,0.086205,0.085874,0.085978,1.304,0.001791,0.084824,-1,Hold
2017-07-14 05:05:00,0.085816,0.085816,0.084824,0.084824,9.770,0.000509,0.084680,-1,Buy
2017-07-14 05:10:00,0.084803,0.084803,0.084608,0.084680,5.446,0.000611,0.086196,1,Sell
2017-07-14 05:15:00,0.084581,0.086196,0.084581,0.086196,8.833,0.000576,0.085496,-1,Buy
2017-07-14 05:20:00,0.085590,0.085810,0.085496,0.085496,4.461,0.000588,0.085850,1,Sell
2017-07-14 05:25:00,0.085496,0.085850,0.085488,0.085850,8.850,0.000587,0.085824,-1,Hold
2017-07-14 05:30:00,0.085850,0.085850,0.085824,0.085824,10.735,0.000586,0.085367,-1,Buy
2017-07-14 05:35:00,0.085938,0.085938,0.085367,0.085367,10.360,0.000594,0.085669,1,Sell


In [309]:
data.Buy_Hold_Sell.value_counts()

Hold    210398
Buy     121592
Sell    121583
Name: Buy_Hold_Sell, dtype: int64